In [345]:
import json
import pandas as pd
import numpy as np
from pprint import pprint
import datetime
data = open("input_csv/ed_trade_data.json")
data = json.load(data)

In [363]:
l = list(data.keys())
l2 = []
for i in l:
    x_i = data[i]
    df_i =  pd.DataFrame(x_i)
    df_i['datetime'] = pd.to_datetime(df_i['date'])
    df_i['price'] = df_i['price'].astype(float)
    df_i['amount'] = df_i['amount'].astype(float)
    df_i['amountBase'] = df_i['amountBase'].astype(float)
    df = df_i[['datetime', 'amount', 'amountBase', 'price']].set_index('datetime')
    pair = i
    row, col = df_i.shape
    maxdate = df.index.max()
    mindate = df.index.min()
    total_days = (maxdate - mindate).days
    total_seconds = (np.datetime64(maxdate) - np.datetime64(mindate)).item().total_seconds()
    total_hours = (total_seconds / 3600)
    grouped = df.groupby(lambda x : x.month)
    grouped.index = 'month'
    a = grouped.count()
    grouped = df.groupby(lambda x: x.month)
    grouped.index = 'month'
    no_of_trades = (grouped['amount'].count())
    a = grouped['amount'].sum()
    b = grouped['amountBase'].sum()
    c = (b / a)
    if (total_days != 0):
        daily_trades_average = sum(no_of_trades) / (total_days)
    else:
        daily_trades_average = 0
    avg_hourly_trades = (sum(no_of_trades) / total_hours)
    result = [pair, row, sum(no_of_trades), str(datetime.timedelta(seconds=total_seconds)), avg_hourly_trades, mindate, maxdate, sum(a), sum(b), (sum(c)/len(c)), total_days]
    l2.append(result)
output = pd.DataFrame(l2)
output.columns = ['pair', 'no_of_rows', 'no_of_trades', 'total_time', 'avg_hourly_trades', 'min_date', 'max_date', 'volume_traded', 'total_turnover', 'avg_price_of_unit_volume', 'total_days']
resultant = output.set_index('pair')
resultant

,no_of_rows,no_of_trades,total_time,avg_hourly_trades,min_date,max_date,volume_traded,total_turnover,avg_price_of_unit_volume,total_days
pair,,,,,,,,,,
ETH_NEWB,353,353,"77 days, 9:32:15",0.190037,2017-08-17 03:59:39,2017-11-02 13:31:54,1.592915e+06,58.522860,3.567360e-05,77
ETH_NTC,500,500,"26 days, 8:46:08",0.790178,2017-10-07 04:18:37,2017-11-02 13:04:45,3.294433e+04,244.158512,6.304795e-03,26
ETH_LNK,271,271,"76 days, 1:48:01",0.148428,2017-08-17 00:23:24,2017-11-01 02:11:25,1.364495e+03,155.122273,1.041670e-01,76
ETH_IXT,500,500,"16 days, 18:21:44",1.242661,2017-10-16 17:33:49,2017-11-02 11:55:33,7.868543e+05,493.627737,6.466308e-04,16
ETH_ETHP,177,177,"15 days, 22:00:22",0.463343,2017-10-17 12:47:02,2017-11-02 10:47:24,5.989485e+05,14.015264,2.582241e-05,15
ETH_SUB,500,500,"8 days, 13:16:56",2.435671,2017-10-24 20:54:45,2017-11-02 10:11:41,8.889812e+05,382.461776,4.114484e-04,8
ETH_1ST,297,297,"76 days, 5:21:59",0.162351,2017-08-18 03:08:49,2017-11-02 08:30:48,9.278350e+04,114.536451,1.856293e-03,76
ETH_ETHG,408,408,"15 days, 18:02:57",1.079225,2017-10-17 09:26:14,2017-11-02 03:29:11,7.730317e+05,66.549733,6.893105e-05,15
ETH_ETHD,500,500,"3 days, 14:02:38",5.810988,2017-10-29 22:14:08,2017-11-02 12:16:46,5.051058e+05,318.079105,6.573749e-04,3


In [324]:
resultant.dtypes

no_of_rows                           int64
min_date                    datetime64[ns]
max_date                    datetime64[ns]
total_days                           int64
no_of_trades                         int64
volume_traded                      float64
total_turnover                     float64
avg_price_of_unit_volume           float64
avg_hourly_trades                  float64
dtype: object

In [356]:
s = resultant.sort_values(by = ['no_of_rows'], ascending = False)
s

,no_of_rows,no_of_trades,total_time,avg_hourly_trades,min_date,max_date,volume_traded,total_turnover,avg_price_of_unit_volume,total_days
pair,,,,,,,,,,
ETH_SONM,500,500,"28 days, 19:33:13",0.723010,2017-10-04 19:40:23,2017-11-02 15:13:36,7.637249e+05,306.836042,3.326063e-04,28
ETH_LRC,500,500,"7 days, 8:38:20",2.830634,2017-10-26 02:10:02,2017-11-02 10:48:22,1.031551e+06,564.507359,5.680508e-04,7
ETH_FC,500,500,"49 days, 13:15:10",0.420432,2017-09-13 21:45:06,2017-11-02 11:00:16,2.066967e+06,183.389347,7.419426e-05,49
ETH_WILD,500,500,"40 days, 20:01:24",0.510192,2017-09-22 20:00:52,2017-11-02 16:02:16,1.128525e+06,365.118367,3.060994e-04,40
ETH_0x08d3,500,500,"41 days, 16:03:21",0.499972,2017-09-21 19:04:54,2017-11-02 11:08:15,4.365061e+08,178.889801,4.205000e-07,41
ETH_HBT,500,500,"10 days, 18:35:12",1.933588,2017-10-22 18:50:15,2017-11-02 13:25:27,1.113948e+06,441.988727,4.121984e-04,10
ETH_CDT,500,500,"25 days, 18:29:42",0.808414,2017-10-07 20:42:45,2017-11-02 15:12:27,2.954183e+06,328.850652,1.098559e-04,25
ETH_BLX,500,500,"58 days, 21:05:28",0.353834,2017-09-04 13:43:41,2017-11-02 10:49:09,1.247101e+05,637.595014,4.950690e-03,58
ETH_ZRX,500,500,"3 days, 21:25:01",5.352348,2017-10-29 18:27:40,2017-11-02 15:52:41,9.120269e+05,546.463359,6.005596e-04,3


In [362]:
s1 = s.query('no_of_trades >= 500')
s2 = s1.query('total_days <= 4')
top_daily_traded_pairs = s2.sort_values(by = ['avg_hourly_trades'], ascending = False)
top_daily_traded_pairs.to_csv('output_files/top__daily_traded_pairs.csv')

In [315]:
def get_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3 - q1 #Interquartile range
    fence_low  = q1 - 1.5 * iqr
    fence_high = q3 + 1.5 * iqr
    df_out = df_in.loc[(df_in[col_name] < fence_low) | (df_in[col_name] > fence_high)]
    return df_out[col_name]
get_outlier(sdata, 'price')


NameError: name 'sdata' is not defined